<center><img src="images/header.png"></center>

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Меры качества кластеризации, уменьшение размерности признаков</center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print u'Так надо'

# Cluster Validity and Quality Measures

### Оценка качества кластеризации при известном groud truth

Пусть $\hat{\pi}$ - это полученное разбиение на кластеры, а $\pi^*$ - ground truth. 

<center><img src='http://cse3521.artifice.cc/images/iris-true-class-sepal.png' width=800></center>


#### Adjusted Rand Index

$$ \text{Rand}(\hat{\pi},\pi^*) = \frac{a + d}{a + b + c + d} \text{,}$$
где 
* $a$ количество пар объектов, находящихся в одинаковых кластерах в $\hat{\pi}$ и
$\pi^*$, 
* $b$ ($c$) количество пар объектов в одном и том же кластере в  $\hat{\pi}$ ($\pi^*$), но в разных в  $\pi^*$ ($\hat{\pi}$)
* $d$ количество пар объектов в разных кластерах в $\hat{\pi}$ и $\pi^*$

Adjusted Rand Index - корректировка Rand index:

$$\text{ARI}(\hat{\pi},\pi^*)   = \frac{\text{Rand}(\hat{\pi},\pi^*) - \text{Expected}}{\text{Max} - \text{Expected}}$$

Так же есть **[Normalized Mutual Information](http://en.wikipedia.org/wiki/Mutual_information)**, но результаты этой метрики схожи с ARI

#### Индекс Жаккара
$$ Jac(\hat{\pi}, \pi^*) = \frac{a}{a + b + c}$$

#### Точность, полнота и F-мера
$$ Precision(\hat{\pi}, \pi^*) = \frac{a}{a + b} $$
$$ Recall(\hat{\pi}, \pi^*) = \frac{a}{a + c} $$
$$ F-measure(\hat{\pi}, \pi^*) = \frac{2Precision \cdot Recall}{Precision + Recall} $$

### Критерий Silhouette

Пусть дана кластеризация в $K$ кластеров, и объект $i$ попал в $C_k$

* $a(i)$ - среднее расстояние от $i$ объекта до объектов из $C_k$
* $b(i) = min_{j \neq k} b_j(i)$,  где $b_j(i)$ - среднее расстояние от $i$ объекта до объектов из $C_j$
$$
silhouette(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$
Средний silhouette для всех точек из $\mathbf{X}$ является критерием качества кластеризации.

<center><img src='images/sil1.png' width=1200></center>

<center><img src='images/sil2.png' width=1200></center>

### Dunn Index

Введем обозначения:
* $\delta(C_k,C_l) = \min\limits_{x_i\in C_k, x_j \in C_l}d(x_i, x_j) $ - расстояние между кластерами $C_k, C_l$
* $\Delta(C_k) = \max\limits_{x_i, x_j \in C_k}d(x_i, x_j)$ - диаметр кластера

Тогда 
$$DI = \frac{\min\limits_{k \neq l} \delta(C_k, C_l)}{\max\limits_{(C_k)} \Delta(C_k)} $$

*Существует несколько вариаций данного индекса*

### Davies–Bouldin Index

Введем обозначения
* $S(C_k) = 1/|C_k|\sum\limits_{x_i \in C_k} d(x_i, \mu_{c_k})$ - разброс данных внутри кластера $C_k$

$$ DB = \frac{1}{K}\sum\limits_{C_k}\max\limits_{C_l \neq C_k}\frac{S(C_K) + S(C_l)}{d(\mu_{c_k}, \mu_{c_l})} $$

*Существует несколько вариаций данного индекса*

### FYI
* [Обзор мер качества 1](http://www.sciencedirect.com/science/article/pii/S003132031200338X)
* [Обзор мер качества 2](https://cran.r-project.org/web/packages/clusterCrit/vignettes/clusterCrit.pdf)

# Методы уменьшения размерности признаков

<center><img src='images/curse.png' width=800></center>

## Проклятье размерности

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">$d=2$<img src='https://jeremykun.files.wordpress.com/2016/01/2d-distances.png' width=400></th>
    <th class="tg-031e">$d=2 \dots 100$<img src='https://jeremykun.files.wordpress.com/2016/01/distances-animation.gif' width=400></th>
  </tr>
</table>

$$ \lim_{d \rightarrow \infty} \frac{\text{dist}_{max} - \text{dist}_{min}}{\text{dist}_{min}} = 0$$

## Проклятье размености

<center><img src='http://www.visiondummy.com/wp-content/uploads/2014/04/curseofdimensionality.png'></center>

## Способы понижения размерности

Избавляться от размерности можно методами **отбора признаков (Feature Selection)** и методами **уменьшения размерности (Feature Reduction)**

### Feature Selection (для Supervised Models)
Методы деляться на следующие группы:
* Filter methods 
    * Признаки рассматриваются независимо друг от друга
    * Изучается индивидуальный "вклад" призника в предсказываемую переменную
    * Быстрое вычисление
    * *Пример?*
* Wrapper methods
    * Идет отбор группы признаков
    * Может быть оооочень медленным, но качество, обычно, лучше чем у Filter Methods
    * Примеры: Stepwise feature selection for regression, [Boruta Algorithm](https://www.google.ru/url?sa=t&rct=j&q=&esrc=s&source=web&cd=2&ved=0ahUKEwif5biy-fTWAhXkYJoKHbdxCLAQFgg2MAE&url=https%3A%2F%2Fwww.jstatsoft.org%2Farticle%2Fview%2Fv036i11%2Fv36i11.pdf&usg=AOvVaw3tyiHN0BCe2fkkAA6xEVDE)
* Embedded methods
    * Отбор признаков "зашит" в модель

#### Filter method - Mutual Information
$$MI(y,x) = \sum_{x,y} p(x,y) \ln\left[\frac{p(x,y)}{p(x)p(y)}\right]$$
Сколько информации $x$ сообщает об $y$.

In [2]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
from sklearn.metrics import mutual_info_score
pd.crosstab(df_titanic.Survived, df_titanic.Sex, normalize=True)

mutual_info_score(df_titanic.Survived, df_titanic.Sex)

0.15087048925218183

#### Wrapper Methods - Recursive Feature Elimination

При данном подходе из линейной модели последовательно удаляются признаки с наименьшим коэффициентом

Выбор итоговой модели можно производить по одному из следующих критериев, оценивающих сложность и точность модели:
* [AIC](https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B9_%D0%90%D0%BA%D0%B0%D0%B8%D0%BA%D0%B5)
* [BIC](https://en.wikipedia.org/wiki/Bayesian_information_criterion)

# Principal Component Analysis
## Метод Главных Компонент

# PCA

* Позволяет уменьшить число переменных, выбрав самые изменчивые из них
* Новые переменные являются линейной комбинацией старых переменных
* Переход к ортономированному базису

<center><img src='http://www.visiondummy.com/wp-content/uploads/2014/05/correlated_2d.png' width=600></center>

## Построение PCA
*  Пусть $x \in \mathbb{R}^d$ - вектор признаков для какого-то объекта. Будем считать, что $x$ - центрировано и отшкалировано. $E[x_i] = 0, V[x_i] = 1, \quad i=1 \dots d$
* Требуется найти линейное преобразование, которое задается ортогональной матрицей $A$:
$$ pc = A^\top x $$

* $pc_i = a_i^\top x = x^\top a_i$
* $cov[x] = E[(x - E[x])(x - E[x])^\top] = Exx^\top = \Sigma$ -  ковариационная матрица

## Построение PCA

* $E[pc_i] = E[a_i^\top x] = a_i^\top E[x]$
* $cov[pc_i, pc_j] = E[pc_i \cdot pc_j^\top] = a_i^\top \Sigma a_j $
* $\Sigma$ - симметричная и положительно определенная матрица.
    * Собственные числа $\lambda_i \in \mathbb{R}, \lambda_i \geq 0$ (Будем считать, что $\lambda_1 \geq \lambda_2  \geq \dots \geq \lambda_d $)
    * Собственные вектора при $\lambda_i \neq \lambda_j $ ортогональны: $v_i^\top v_j = 0$
    * У каждого $\lambda_i$ есть единственный $v_i$

## Первая компонента
$$ pc_1 = a_1 ^\top x $$

\begin{equation}
\begin{cases}
V[pc_1] = a_1^\top \Sigma a_1 \rightarrow \max_a \\
a_1^\top a_1 = 1
\end{cases}
\end{equation}
* Строим функцию лагранжа
$$ \mathcal{L}(a_1, \nu) = a_1^\top \Sigma a_1 - \nu (a_1^\top a_1 - 1) \rightarrow max_{a_1, \nu}$$
* Считаем производую по $a_1$
$$ \frac{\partial\mathcal{L}}{\partial a_1} = 2\Sigma a_1 - 2\nu a_1 = 0 $$
* Получается, что $a_1$ один из собственных векторов матрицы $\Sigma$, причем при $\lambda_1$
$$ V[pc_1] = a_1^\top \Sigma a_1 = \lambda_i a_1^\top a_1 = \lambda_i $$

## Вторая компонента
$$ pc_2 = a_2 ^\top x $$

\begin{equation}
\begin{cases}
V[pc_2] = a_2^\top \Sigma a_2 \rightarrow \max_a \\
a_2^\top a_2 = 1 \\
cov[pc_1, pc_2] = a_2^\top \Sigma a_1 = \lambda_1 a_2^\top a_1 = 0
\end{cases}
\end{equation}
* Строим функцию лагранжа
$$ \mathcal{L}(a_2, \nu, \tau) = a_2^\top \Sigma a_2 - \nu (a_2^\top a_2 - 1) - \tau a_2^\top a_1 \rightarrow max_{a_1, \nu}$$

Аналогичными выкладками приходим к тому, что $a_2$ - собственный вектор $\Sigma$ при $\lambda_2$

и т.д.

## Singular Value Decomposition

Для любой матрицы $X$ размера $n \times m$ и ранга $r$ можно найти разложение вида:
$$ X = U S V^\top ,$$
где 
* $U$ - унитарная матрица, состоящая из собственных векторов $XX^\top$
* $V$ - унитарная матрица, состоящая из собственных векторов $X^\top X$
* $S$ - диагональная матрица с сингулярными числами $s_i = \sqrt{\lambda_i}$

## SVD
Матрицы $U$ и $V$ ортогональны и могут быть использованы для перехода к ортогональному базису:
$$ XV = US $$

Сокращение размерности заключается в том, что вместо того, чтобы умножать $X$ не на всю матрицу $V$, а лишь на первые $k<m$ её столбцов - матрицу $V'$

Квадраты сингулярных чисел в $S$ содержат дисперсию, объясненную в главных компонентах

<center><img src='images/pca_svd.png' width=600></center>

## MNIST, DIGITS
<center><img src='images/digits.png' width=600></center>

## MNIST PCA

<center><img src='http://nikhilbuduma.com/img/autoencoder_digit_exp.png' width=1200></center>

## Резюме

* PCA понижает размерность признакового пространства
* Новые компоненты являются линейной комбинацией исходных признаков
* Новые компоненты - ортогональны
* Можно применять в моделях и для визуализации
* [PCA SVD туториал](https://www.cs.princeton.edu/picasso/mats/PCA-Tutorial-Intuition_jp.pdf)

# t-SNE
## t-distributed stochastic neighbor embedding

## Идея

* Перейти в пространство меньшей размерности так, чтобы расстояния между объектами в новом пространстве были подобны расстояниям в исходном пространстве.
* Это многомерное шкалирование!

<center><img src='images/mds.png' width =1200></center>

* t-SNE - почти про это, но вместо этого мы будем пытаться перенести окрестность точек из исходного пространства в пространоство меньшей размерности.

* Схожесть между объектами в исходном пространстве $\mathbb{R}^m$
$$
p(i, j) = \frac{p(i | j) + p(j | i)}{2n}, \quad p(j | i) = \frac{\exp(-\|\mathbf{x}_j-\mathbf{x}_i\|^2/{2 \sigma_i^2})}{\sum_{k \neq i}\exp(-\|\mathbf{x}_k-\mathbf{x}_i\|^2/{2 \sigma_i^2})}
$$
$\sigma_i$ неявно задается пользователем
* Схожесть между объектами в целевом пространстве $\mathbb{R}^k, k << m$
$$
q(i, j) = \frac{g(|\mathbf{y}_i - \mathbf{y}_j|)}{\sum_{k \neq l} g(|\mathbf{y}_i - \mathbf{y}_j|)}
$$ 
где $g(z) = \frac{1}{1 + z^2}$ - распределение Коши (t-распределение Стьюдента с 1 степенью свободы)
* Критерий
$$
J_{t-SNE}(y) = KL(P \| Q) = \sum_i \sum_j p(i, j) \log \frac{p(i, j)}{q(i, j)} \rightarrow \min\limits_{\mathbf{y}}
$$

## Дивергенция Кульбака-Лейблера

* Насколько распределение $P$ отличается от распределения $Q$?
$$
KL(P \| Q) = \sum_z P(z) \log \frac{P(z)}{Q(z)}
$$

<center><img src='http://www.science-emergence.com/media/images/381.png' width=1200></center>

## Оптимизация

* Оптимизируем $J_{t-SNE}(y)$ с помощью градиентного спуска

$$\frac{\partial J_{t-SNE}}{\partial y_i}=4 \sum_j(p(i,j)−q(i,j))(y_i−y_j)g(|y_i−y_j|)$$

* [Статья](http://jmlr.csail.mit.edu/papers/volume9/vandermaaten08a/vandermaaten08a.pdf)
* [Примеры](http://lvdmaaten.github.io/tsne/)
* [Демо и советы](http://distill.pub/2016/misread-tsne/)
    * t-SNE может быть нестабильным
    * Размеры полученных сгустков могут ничего не значить
    * Расстояния между кластерами могут ничего не значить
    * Полностью шумовые данные могут выдать структуру

<center><img src='http://lvdmaaten.github.io/tsne/examples/mnist_tsne.jpg' width=500></center>

<center><img src='images/mainfold.png'></center>

# Вопросы?

## Пожалуйста, оставьте отзыв о лекции